# Deep Learning Grid Search

In this project, we will learn how to use the scikit-learn grid search capability.

We are going to learn the following topics:

* How to use Keras models in scikit-learn.
* How to use grid search in scikit-learn.
* How to tune batch size and training epochs.
* How to tune learning rate
* How to tune network weight initialization.
* How to tune activation functions.
* How to tune dropout regularization.
* How to tune the number of neurons in the hidden layer.

In [18]:
import sys
import pandas
import numpy
import sklearn
import keras

print('Python: {}'.format(sys.version))
print('Pandas: {}'.format(pandas.__version__))
print('Numpy: {}'.format(numpy.__version__))
print('Sklearn: {}'.format(sklearn.__version__))
print('Keras: {}'.format(keras.__version__))

Python: 2.7.16 |Anaconda, Inc.| (default, Mar 14 2019, 21:00:58) 
[GCC 7.3.0]
Pandas: 0.24.2
Numpy: 1.16.2
Sklearn: 0.18
Keras: 2.2.4


In [19]:
import pandas as pd
import numpy as np

# import the uci pima indians diabetes dataset
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
names = ['n_pregnant', 'glucose_concentration', 'blood_pressuer (mm Hg)', 'skin_thickness (mm)', 'serum_insulin (mu U/ml)',
        'BMI', 'pedigree_function', 'age', 'class']
df = pd.read_csv(url, names = names)

In [20]:
# Describe the dataset
df.describe()

,n_pregnant,glucose_concentration,blood_pressuer (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [21]:
df[df['glucose_concentration'] == 0]

,n_pregnant,glucose_concentration,blood_pressuer (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
75,1,0,48,20,0,24.7,0.140,22,0
182,1,0,74,20,23,27.7,0.299,21,0
342,1,0,68,35,0,32.0,0.389,22,0
349,5,0,80,32,0,41.0,0.346,37,1
502,6,0,68,41,0,39.0,0.727,41,1


In [22]:
# Preprocess the data, mark zero values as NaN and drop
columns = ['glucose_concentration', 'blood_pressuer (mm Hg)', 'skin_thickness (mm)', 'serum_insulin (mu U/ml)', 'BMI']

for col in columns:
    df[col].replace(0, np.NaN, inplace=True)
    
df.describe()

,n_pregnant,glucose_concentration,blood_pressuer (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
count,768.000000,763.000000,733.000000,541.000000,394.000000,757.000000,768.000000,768.000000,768.000000
mean,3.845052,121.686763,72.405184,29.153420,155.548223,32.457464,0.471876,33.240885,0.348958
std,3.369578,30.535641,12.382158,10.476982,118.775855,6.924988,0.331329,11.760232,0.476951
min,0.000000,44.000000,24.000000,7.000000,14.000000,18.200000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,64.000000,22.000000,76.250000,27.500000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,29.000000,125.000000,32.300000,0.372500,29.000000,0.000000
75%,6.000000,141.000000,80.000000,36.000000,190.000000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [23]:
# Drop rows with missing values
df.dropna(inplace=True)

# summarize the number of rows and columns in df
df.describe()

,n_pregnant,glucose_concentration,blood_pressuer (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,3.301020,122.627551,70.663265,29.145408,156.056122,33.086224,0.523046,30.864796,0.331633
std,3.211424,30.860781,12.496092,10.516424,118.841690,7.027659,0.345488,10.200777,0.471401
min,0.000000,56.000000,24.000000,7.000000,14.000000,18.200000,0.085000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,21.000000,76.750000,28.400000,0.269750,23.000000,0.000000
50%,2.000000,119.000000,70.000000,29.000000,125.500000,33.200000,0.449500,27.000000,0.000000
75%,5.000000,143.000000,78.000000,37.000000,190.000000,37.100000,0.687000,36.000000,1.000000
max,17.000000,198.000000,110.000000,63.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [24]:
# Convert dataframe to numpy array
dataset = df.values
print(dataset.shape)

(392, 9)


In [25]:
# split into input (X) and an output (Y)
X = dataset[:,0:8]
Y = dataset[:, 8].astype(int)

In [26]:
print(X.shape)
print(Y.shape)
print(Y[:5])

(392, 8)
(392,)
[0 1 1 1 1]


In [27]:
# Normalize the data using sklearn StandardScaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X)

In [28]:
print(scaler)

StandardScaler(copy=True, with_mean=True, with_std=True)


In [29]:
# Transform and display the training data
X_standardized = scaler.transform(X)

data = pd.DataFrame(X_standardized)
data.describe()

,0,1,2,3,4,5,6,7
count,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02
mean,-4.021726e-17,3.129583e-17,-4.641624e-16,1.042250e-16,6.485742e-17,1.543550e-16,3.880116e-17,1.028089e-16
std,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00
min,-1.029213e+00,-2.161731e+00,-3.739001e+00,-2.108484e+00,-1.196867e+00,-2.120941e+00,-1.269525e+00,-9.682991e-01
25%,-7.174265e-01,-7.665958e-01,-6.941640e-01,-7.755315e-01,-6.681786e-01,-6.676780e-01,-7.340909e-01,-7.719850e-01
50%,-4.056403e-01,-1.176959e-01,-5.314565e-02,-1.384444e-02,-2.574448e-01,1.621036e-02,-2.131475e-01,-3.793569e-01
75%,5.297185e-01,6.609841e-01,5.878727e-01,7.478426e-01,2.859877e-01,5.718696e-01,4.751644e-01,5.040564e-01
max,4.271153e+00,2.445459e+00,3.151946e+00,3.223325e+00,5.812990e+00,4.846172e+00,5.497667e+00,4.921123e+00


In [30]:
# import necessary sklearn and keras packages
from sklearn.model_selection import GridSearchCV, KFold
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam

In [31]:
# Start defining the model
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(4, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = 0.01)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

model = create_model()
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_88 (Dense)             (None, 8)                 72        
_________________________________________________________________
dense_89 (Dense)             (None, 4)                 36        
_________________________________________________________________
dense_90 (Dense)             (None, 1)                 5         
Total params: 113
Trainable params: 113
Non-trainable params: 0
_________________________________________________________________
None


In [32]:
# Do a grid search for the optimal batch size and number of epochs
# import necessary packages
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam

# Define a random seed
seed = 6
np.random.seed(seed)

# Start defining the model
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(4, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = 0.01)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, verbose = 1)

# define the grid search parameters
batch_size = [10, 20, 40]
epochs = [10, 50, 100]

# make a dictionary of the grid search parameters
param_grid = dict(batch_size=batch_size, epochs=epochs)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] epochs=10, batch_size=10 ........................................
Epoch 1/10
261/261 [==============================] - 0s 48us/step - loss: 0.6333 - acc: 0.6897
Epoch 2/10
261/261 [==============================] - 0s 43us/step - loss: 0.4840 - acc: 0.6973
Epoch 3/10
261/261 [==============================] - 0s 44us/step - loss: 0.4422 - acc: 0.6973
Epoch 4/10
261/261 [==============================] - 0s 42us/step - loss: 0.4246 - acc: 0.6973
Epoch 5/10
261/261 [==============================] - 0s 44us/step - loss: 0.4142 - acc: 0.8123
Epoch 6/10
261/261 [==============================] - 0s 46us/step - loss: 0.4022 - acc: 0.8276
Epoch 7/10
261/261 [==============================] - 0s 44us/step - loss: 0.3971 - acc: 0.8352
Epoch 8/10
261/261 [==============================] - 0s 41us/step - loss: 0.3850 - acc: 0.8276
Epoch 9/10
261/261 [==============================] - 0s 43us/step - loss: 0.3844 - acc: 0.8199
Epoch

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.4s remaining:    0.0s


Epoch 1/10
261/261 [==============================] - 0s 41us/step - loss: 0.6607 - acc: 0.7280
Epoch 2/10
261/261 [==============================] - 0s 43us/step - loss: 0.6058 - acc: 0.7816
Epoch 3/10
261/261 [==============================] - 0s 44us/step - loss: 0.5757 - acc: 0.7854
Epoch 4/10
261/261 [==============================] - 0s 42us/step - loss: 0.5311 - acc: 0.8123
Epoch 5/10
261/261 [==============================] - 0s 43us/step - loss: 0.5161 - acc: 0.8008
Epoch 6/10
261/261 [==============================] - 0s 43us/step - loss: 0.5176 - acc: 0.7969
Epoch 7/10
261/261 [==============================] - 0s 46us/step - loss: 0.5040 - acc: 0.7969
Epoch 8/10
261/261 [==============================] - 0s 50us/step - loss: 0.4814 - acc: 0.7931
Epoch 9/10
261/261 [==============================] - 0s 44us/step - loss: 0.4738 - acc: 0.8008
Epoch 10/10
261/261 [==============================] - 0s 16us/step
[CV] ............... epochs=10, batch_size=10, score=0.770992 -   0.

[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.8s remaining:    0.0s


Epoch 1/10
262/262 [==============================] - 0s 41us/step - loss: 0.6352 - acc: 0.6450
Epoch 2/10
262/262 [==============================] - 0s 43us/step - loss: 0.5274 - acc: 0.6527
Epoch 3/10
262/262 [==============================] - 0s 40us/step - loss: 0.5014 - acc: 0.6908
Epoch 4/10
262/262 [==============================] - 0s 40us/step - loss: 0.4988 - acc: 0.7595
Epoch 5/10
262/262 [==============================] - 0s 40us/step - loss: 0.4885 - acc: 0.7786
Epoch 6/10
262/262 [==============================] - 0s 41us/step - loss: 0.4830 - acc: 0.7824
Epoch 7/10
262/262 [==============================] - 0s 43us/step - loss: 0.4775 - acc: 0.7824
Epoch 8/10
262/262 [==============================] - 0s 41us/step - loss: 0.4745 - acc: 0.7786
Epoch 9/10
262/262 [==============================] - 0s 43us/step - loss: 0.4702 - acc: 0.7939
Epoch 10/10
262/262 [==============================] - 0s 23us/step
[CV] ............... epochs=10, batch_size=10, score=0.823077 -   0.

...

262/262 [==============================] - 0s 22us/step - loss: 0.4399 - acc: 0.7786
Epoch 18/100
262/262 [==============================] - 0s 20us/step - loss: 0.4387 - acc: 0.7748
Epoch 19/100
262/262 [==============================] - 0s 20us/step - loss: 0.4348 - acc: 0.7863
Epoch 20/100
262/262 [==============================] - 0s 23us/step - loss: 0.4349 - acc: 0.7863
Epoch 21/100
262/262 [==============================] - 0s 20us/step - loss: 0.4335 - acc: 0.7824
Epoch 22/100
262/262 [==============================] - 0s 20us/step - loss: 0.4260 - acc: 0.7786
Epoch 23/100
262/262 [==============================] - 0s 20us/step - loss: 0.4274 - acc: 0.7901
Epoch 24/100
262/262 [==============================] - 0s 19us/step - loss: 0.4269 - acc: 0.7939
Epoch 25/100
262/262 [==============================] - 0s 22us/step - loss: 0.4211 - acc: 0.7901
Epoch 26/100
262/262 [==============================] - 0s 18us/step - loss: 0.4206 - acc: 0.7901
Epoch 27/100
262/262 [===========

262/262 [==============================] - 0s 9us/step
[CV] .............. epochs=100, batch_size=40, score=0.830769 -   0.2s


[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:   50.8s finished


Epoch 1/10
392/392 [==============================] - 0s 23us/step - loss: 0.6724 - acc: 0.6480
Epoch 2/10
392/392 [==============================] - 0s 30us/step - loss: 0.5814 - acc: 0.6684
Epoch 3/10
392/392 [==============================] - 0s 33us/step - loss: 0.5102 - acc: 0.6684
Epoch 4/10
392/392 [==============================] - 0s 30us/step - loss: 0.4888 - acc: 0.6684
Epoch 5/10
392/392 [==============================] - 0s 25us/step - loss: 0.4735 - acc: 0.6684
Epoch 6/10
392/392 [==============================] - 0s 30us/step - loss: 0.4651 - acc: 0.7628
Epoch 7/10
392/392 [==============================] - 0s 32us/step - loss: 0.4599 - acc: 0.7985
Epoch 8/10
392/392 [==============================] - 0s 30us/step - loss: 0.4525 - acc: 0.7959
Epoch 9/10
392/392 [==============================] - 0s 28us/step - loss: 0.4458 - acc: 0.7985
Epoch 10/10
392/392 [==============================] - 0s 24us/step - loss: 0.4393 - acc: 0.7985
Best: 0.785714291796, using {'epochs': 

NameError: name 'grid_result' is not defined

In [ ]:
# Do a grid search for learning rate and dropout rate
# import necessary packages
from keras.layers import Dropout

# Define a random seed
seed = 6
np.random.seed(seed)

# Start defining the model
def create_model(learn_rate, dropout_rate):
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(4, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = learn_rate)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, epochs = 100, batch_size = 20, verbose = 0)

# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1]
dropout_rate = [0.0, 0.1, 0.2]

# make a dictionary of the grid search parameters
param_grid = dict(learn_rate=learn_rate, dropout_rate=dropout_rate)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

In [ ]:
# Do a grid search to optimize kernel initialization and activation functions
# import necessary packages

# Define a random seed
seed = 6
np.random.seed(seed)

# Start defining the model
def create_model(activation, init):
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer= init, activation= activation))
    model.add(Dense(4, input_dim = 8, kernel_initializer= init, activation= activation))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = 0.001)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, epochs = 100, batch_size = 20, verbose = 0)

# define the grid search parameters
activation = ['softmax', 'relu', 'tanh', 'linear']
init = ['uniform', 'normal', 'zero']

# make a dictionary of the grid search parameters
param_grid = dict(activation = activation, init = init)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

In [ ]:
# Do a grid search to find the optimal number of neurons in each hidden layer
# import necessary packages

# Define a random seed
seed = 6
np.random.seed(seed)

# Start defining the model
def create_model(neuron1, neuron2):
    # create model
    model = Sequential()
    model.add(Dense(neuron1, input_dim = 8, kernel_initializer= 'uniform', activation= 'linear'))
    model.add(Dense(neuron2, input_dim = neuron1, kernel_initializer= 'uniform', activation= 'linear'))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = 0.001)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, epochs = 100, batch_size = 20, verbose = 0)

# define the grid search parameters
neuron1 = [4, 8, 16]
neuron2 = [2, 4, 8]

# make a dictionary of the grid search parameters
param_grid = dict(neuron1 = neuron1, neuron2 = neuron2)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), refit = True, verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

In [ ]:
# generate predictions with optimal hyperparameters
y_pred = grid.predict(X_standardized)

In [ ]:
print(y_pred.shape)

In [ ]:
print(y_pred[:5])

In [ ]:
# Generate a classification report
from sklearn.metrics import classification_report, accuracy_score

print(accuracy_score(Y, y_pred))
print(classification_report(Y, y_pred))